In [47]:
import numpy as np
# Train a model to classify real pulses vs. noise
import tensorflow as tf
from tensorflow.keras.layers import Input,  Dense
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
import awkward as ak
from tensorflow.keras.layers import Dropout
from tensorflow import keras
from tensorflow.keras import layers
import random

In [48]:
num_events = 50000
num_samples = 10
xmin = 1
xmax = 10

In [179]:
def plot_timesamples(ax, time, emsignal_list, noisedata_list):
    #fig, ax = plt.subplots()
    for i, emsignal in enumerate(emsignal_list):
        noisedata = noisedata_list[i][1:]
        ax.plot(time, emsignal[1:], label=f'EM pulse' if i==1 else None, color='blue')
        ax.plot(time, noisedata, label=f'noise'  if i==1 else None,color='orange')
    ax.set_xlabel('Time (ns)')
    ax.set_ylabel('Amplitude (arb. units)')
    ax.legend()

In [180]:
import uproot
### Energy = 0.15 GeV
root_file = uproot.open(f"/eos/user/d/dasgupsu/SWAN_projects/ECAL_noise_EM_discrimination/data/outputPSWithNoPU_withNoise_0.150000_1.0_v1.root")
tree = root_file["Samples"]
arrays = tree.arrays(["samples", "ysamples","samplesNoise","ysamplesNoise", "waveforms"])
X_real = ak.to_numpy(arrays["samples"])
y_real = ak.to_numpy(arrays["ysamples"])
X_noise = ak.to_numpy(arrays["samplesNoise"])
y_noise = ak.to_numpy(arrays["ysamplesNoise"])
X_waveform = ak.to_numpy(arrays["waveforms"])
X_Zero = np.zeros((X_waveform.shape[0], X_waveform.shape[1]), dtype=float)

print(X_real) 
print("Printing y_real")
print(y_real)

#data = np.concatenate([X_real, X_noise]) ### makes it [2*num_events,num_samples]
data = X_real
#labels = np.concatenate([y_real,y_noise])
labels = y_real
#X_target = np.concatenate([X_waveform, X_Zero])
X_target = X_waveform
# Shuffle data and labels together
## Important to shuffle since I take some fraction of events so it should not happen that all the real events 
## are cluttered at the beginning

shuffle_indices = np.random.permutation(len(data))
data = data[shuffle_indices]
labels = labels[shuffle_indices]
X_target = X_target[shuffle_indices]

#fig, ax = plt.subplots()
import awkward as ak

num_events = ak.num(X_real, axis=0)
t_events = np.zeros((num_events, num_samples))
for i in range(num_events):
    t_event = np.linspace(xmin, xmax, 10)
    t_events[i] = t_event
###replace this block with actual time variable

#fig, ax = plt.subplots()
#ev_numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9 , 10, 100, 200, 300]# list of event numbers to plot
ev_numbers = [1500]
emsignal_list = [X_real[ev_number] for ev_number in ev_numbers]
noisedata_list = [X_noise[ev_number] for ev_number in ev_numbers]
#plot_timesamples(ax, t_events[ev_numbers[0]], emsignal_list, noisedata_list)
#plt.show()

[[-0.02302645 -0.01106236 -0.04128678 ...  0.43723053  0.23318535
   0.12932118]
 [-0.15864496 -0.07084266  0.00636395 ...  0.49423221  0.3581044
   0.25740232]
 [ 0.05790091  0.03022707  0.09363004 ...  0.92370825  0.81255649
   0.68655397]
 ...
 [-0.05372396 -0.17839269 -0.17153385 ...  0.37079516  0.16143717
   0.06845977]
 [ 0.13169355  0.31387656  0.36568349 ...  0.7599437   0.6174581
   0.39059257]
 [-0.20073302 -0.15702521 -0.02750234 ...  0.45211461  0.18901024
   0.06145787]]
Printing y_real
[[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]


In [181]:
import awkward as ak

num_events_data = ak.num(data, axis=0)
print(f'number of rows in data is {num_events_data}')


ntimeSamples_data = ak.num(data, axis=1)
print(f'number of rows in data is {ntimeSamples_data[0]}') ## just take the 0th event

# Split into train and test sets
train_size = int(0.7 * num_events_data) ###times 2 because the noise is also in the same dataset, so it is 2*num_events
train_data = data[:train_size]
train_labels = labels[:train_size]
train_target = X_target[:train_size]
test_data = data[train_size:]
test_labels = labels[train_size:]
test_target = X_target[train_size:]
print(f'Size of training data is {train_size}')

'''
print(train_size)
print(train_data)
print(test_data)
'''

print(f'number of elements in data : training data : test data : test target : train target: {len(data)} : {len(train_data)} : {len(test_data)}: {len(test_target)}: {len(train_target)}')

number of rows in data is 50000
number of rows in data is 11
Size of training data is 35000
number of elements in data : training data : test data : test target : train target: 50000 : 35000 : 15000: 15000: 35000


In [182]:
train_data[0]

array([-0.08832286, -0.11586289, -0.10912243,  0.0039692 ,  0.63603957,
        1.08951627,  1.06605966,  0.91080929,  0.74769153,  0.55507618,
        0.36264929])

In [183]:
x_train = train_data
y_train = train_labels 
x_test = test_data
y_test = test_labels

In [184]:
#train_target = train_target.reshape((train_target.shape[0], train_target.shape[1], 1))
#test_target = test_target.reshape((test_target.shape[0], test_target.shape[1], 1))
train_target = train_target.reshape((train_target.shape[0], train_target.shape[1]))
test_target = test_target.reshape((test_target.shape[0], test_target.shape[1]))

In [187]:
train_target[0].shape

(11,)

In [142]:
#x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
#x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

In [175]:
start_token = 99.*np.ones((x_train.shape[0],1))
end_token = -99.*np.ones((x_train.shape[0],1))

In [144]:
y_train = y_train.reshape((y_train.shape[0],1))
y_test = y_test.reshape((y_test.shape[0],1))

In [177]:
train_target = np.concatenate((start_token, train_target, end_token ), axis=0)
#test_target = np.concatenate((start_token, test_target, end_token ), axis=1)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 3 dimension(s)

In [146]:
train_target[:, :-2][0]

array([[99.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.5417028 ],
       [ 0.96375969],
       [ 0.96371202],
       [ 0.78764783],
       [ 0.58232425],
       [ 0.40582712]])

In [147]:
train_target[:, 1:-1][3]

array([[0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.5417028 ],
       [0.96375969],
       [0.96371202],
       [0.78764783],
       [0.58232425],
       [0.40582712],
       [0.27246438]])

In [148]:
transformer_train = {"encoder_inputs": x_train, "decoder_inputs": train_target[:, :-2]}

In [149]:
train_dataset = tf.data.Dataset.from_tensor_slices((transformer_train, train_target[:, 1:-1]))

In [150]:
train_dataset = train_dataset.batch(64)

In [151]:
train_dataset

<BatchDataset element_spec=({'encoder_inputs': TensorSpec(shape=(None, 11, 1), dtype=tf.float64, name=None), 'decoder_inputs': TensorSpec(shape=(None, 11, 1), dtype=tf.float64, name=None)}, TensorSpec(shape=(None, 11, 1), dtype=tf.float64, name=None))>

In [165]:
class Time2Vec(layers.Layer):

    def __init__(self, kernel, activation):
        super(Time2Vec, self).__init__()
        if activation in ["sin", "cos"]:
            activation = {"sin": tf.math.sin, "cos": tf.math.cos}[activation]

        self.k = kernel - 1
        self.p_activation = activation

    def build(self, input_shape):

        # Linear component
        self.w_b = self.add_weight(
            shape=(1, input_shape[1], 1), initializer="uniform", trainable=True
        )

        self.b_b = self.add_weight(
            shape=(1, input_shape[1], 1), initializer="uniform", trainable=True
        )

        # Periodic components
        self.freq = self.add_weight(
            shape=(1, input_shape[1], self.k), initializer="uniform", trainable=True
        )

        self.phase = self.add_weight(
            shape=(1, input_shape[1], self.k), initializer="uniform", trainable=True
        )

        #super().build(input_shape)

    def call(self, inputs: tf.Tensor, **kwargs) -> tf.Tensor:

        inputs = tf.expand_dims(inputs, axis=-1) #(batch_size, feature_size, 1)

        # Linear components
        lin = (
            # Multiply each time dimension with the corresponding linear time component
            tf.multiply(inputs, self.w_b)
            # Bias component for each time dimension
            + self.b_b
        )

        # Periodic components
        # Multiply each time dimension (M, D, H, mins, etc.) with the corresponding frequency vector
        per = tf.multiply(tf.tile(inputs, multiples=[1, 1, self.k]), self.freq)
        # Phase vector for each time dimension
        per = self.p_activation(per + self.phase)
        return tf.concat([lin, per], -1)

    def compute_output_shape(self, input_shape: tuple) -> tuple:
       
        return (input_shape[0], input_shape[1], self.k + 1)

In [166]:
def get_angles(pos, k, d):
    i = k // 2
    angles = pos / np.power(10000, 2 * i / d)

    return angles

In [167]:
def positional_encoding(positions, d):

    angle_rads = get_angles(np.arange(positions)[:, np.newaxis],
                            np.arange(d)[np.newaxis, :],
                            d)
  
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
  
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)


In [168]:
from tensorflow import keras
from tensorflow.keras import layers

In [169]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = inputs
    #x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    #x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    #x = layers.LayerNormalization(epsilon=1e-6)(res)
    #x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    #x = layers.Dropout(dropout)(x)
    #x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return  res

In [170]:
def get_causal_attention_mask(inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)

In [171]:
def transformer_decoder(x, enc_output, head_size, num_heads, ff_dim, dropout=0, mask=None):
    causal_mask = get_causal_attention_mask(x)
    
    # Normalization and Attention
    #x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    mult_attn_out1 = layers.MultiHeadAttention(
                            key_dim=head_size, num_heads=num_heads, dropout=dropout
            )(x, x, x, attention_mask=causal_mask)
    
    #Q1 = layers.LayerNormalization(epsilon=1e-6)(mult_attn_out1 + x)
    Q1 = mult_attn_out1 
    mult_attn_out2 = layers.MultiHeadAttention(
                            key_dim=head_size, num_heads=num_heads, dropout=dropout
            )(Q1, enc_output, enc_output)
    #Q2 = layers.LayerNormalization(epsilon=1e-6)(mult_attn_out2 + Q1)
    Q2 = mult_attn_out2 + Q1
    # Feed Forward Part
    #x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(Q2)
    #x = layers.Dropout(dropout)(x)
    #x = layers.Conv1D(filters=1, kernel_size=1)(x)
    #out3 = x + mult_attn_out2
    return Q2

In [172]:
#layers.MultiHeadAttention?

In [173]:
def build_model(
        input_shape,
        head_size,
        num_heads,
        ff_dim,
        num_transformer_blocks,
        mlp_units,
        dropout=0,
        mlp_dropout=0,
    ):

    embed_dim = 64
    encoder_inputs = keras.Input(shape=input_shape, name="encoder_inputs")
    x = encoder_inputs
    x = Time2Vec(64, "sin")(x)
    #x += positional_encoding(input_shape[0],1)
    for _ in range(num_transformer_blocks):
            x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)
    encoder_outputs = x
    encoder = keras.Model(encoder_inputs, encoder_outputs)

    decoder_inputs = keras.Input(shape=((input_shape[0]),input_shape[1]), name="decoder_inputs")
    encoded_seq_inputs = keras.Input(shape=(input_shape[0], embed_dim), name="decoder_state_inputs")
    x = decoder_inputs
    x += Time2Vec(64, "sin")(x)
    for _ in range(num_transformer_blocks):
            x = transformer_decoder(x, encoded_seq_inputs, head_size, num_heads, ff_dim, dropout)
    decoder_outputs = x
    decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)
    decoder_outputs = decoder([decoder_inputs, encoder_outputs])
    
    transformer = keras.Model(
        [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
    )
    return transformer

In [174]:
input_shape = x_train.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)
model.summary(expand_nested=True)

#opt = SGD(lr=0.01, momentum=0.9)
opt = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

history = model.fit(
    train_dataset,
    epochs=15,
    #callbacks=callbacks,
)


ValueError: Exception encountered when calling layer "time2_vec" (type Time2Vec).

in user code:

    File "/tmp/ipykernel_408/3782225129.py", line 47, in call  *
        per = tf.multiply(tf.tile(inputs, multiples=[1, 1, self.k]), self.freq)

    ValueError: Shape must be rank 4 but is rank 3 for '{{node time2_vec/Tile}} = Tile[T=DT_FLOAT, Tmultiples=DT_INT32](time2_vec/ExpandDims, time2_vec/Tile/multiples)' with input shapes: [?,11,1,1], [3].


Call arguments received:
  • inputs=tf.Tensor(shape=(None, 11, 1), dtype=float32)
  • kwargs={'training': 'False'}

In [161]:
train_dataset    

<BatchDataset element_spec=({'encoder_inputs': TensorSpec(shape=(None, 11, 1), dtype=tf.float64, name=None), 'decoder_inputs': TensorSpec(shape=(None, 11, 1), dtype=tf.float64, name=None)}, TensorSpec(shape=(None, 11, 1), dtype=tf.float64, name=None))>

In [162]:
def decode_sequence(input_sentence, target_sentence):
    tokenized_input_sentence = tf.convert_to_tensor(input_sentence)
    #decoded_sentence = np.zeros((input_sentence.shape[1]-1, 1))[ np.newaxis, ...]
    #decoded_sentence = np.concatenate((99.0*np.ones((1,1,1)), decoded_sentence), axis=1)
    decoded_sentence = np.reshape(target_sentence,(1,11,1))
    """
    for i in range(1):
        tokenized_target_sentence = tf.convert_to_tensor(decoded_sentence)
        predictions = model([tokenized_input_sentence, tokenized_target_sentence])
        #print("suman",predictions)
        if i <10:
            decoded_sentence[0, i+1, 0] = predictions[0, i, 0]
            #print("suman1",decoded_sentence)
    """
    tokenized_target_sentence = tf.convert_to_tensor(decoded_sentence)
    predictions = model([tokenized_input_sentence, tokenized_target_sentence])
    return decoded_sentence


In [163]:
trainX = [np.transpose(pair)[..., np.newaxis] for pair in x_train]

In [164]:
i = 0
for _ in range(4):
    input_sentence = trainX[i]
    target_sentence = train_target[i][:-2]
    translated = decode_sequence(input_sentence, target_sentence)
    print(translated)
    print(train_target[i])
    print(i)
    i +=1

[[[99.        ]
  [ 0.        ]
  [ 0.        ]
  [ 0.        ]
  [ 0.        ]
  [ 0.5417028 ]
  [ 0.96375969]
  [ 0.96371202]
  [ 0.78764783]
  [ 0.58232425]
  [ 0.40582712]]]
[[ 99.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.5417028 ]
 [  0.96375969]
 [  0.96371202]
 [  0.78764783]
 [  0.58232425]
 [  0.40582712]
 [  0.27246438]
 [-99.        ]]
0
[[[99.        ]
  [ 0.        ]
  [ 0.        ]
  [ 0.        ]
  [ 0.        ]
  [ 0.5417028 ]
  [ 0.96375969]
  [ 0.96371202]
  [ 0.78764783]
  [ 0.58232425]
  [ 0.40582712]]]
[[ 99.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.5417028 ]
 [  0.96375969]
 [  0.96371202]
 [  0.78764783]
 [  0.58232425]
 [  0.40582712]
 [  0.27246438]
 [-99.        ]]
1
[[[99.        ]
  [ 0.        ]
  [ 0.        ]
  [ 0.        ]
  [ 0.        ]
  [ 0.5417028 ]
  [ 0.96375969]
  [ 0.96371202]
  [ 0.78764783]
  [ 0.58232425]
  [ 0.40582712]]]
[[ 99.        ]
 [  0.        ]
 [  0.      